# California Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [1]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [ ]:
# review of robots.txt - no restrictions

## Scrape Links
`Selenium`

### Try for one link

In [8]:
# Access webpage
link = 'https://www.gov.ca.gov/2023/12/'
server_response = requests.get(link, header_list)
server_response
# response 403 
# Beautiful Soup doesn't seem to work

<Response [403]>

In [11]:
# try selenium 
driver = webdriver.Chrome()
driver.get('https://www.gov.ca.gov/2023/12/')

#wait = WebDriverWait(driver, 3)
results = driver.find_elements(By.TAG_NAME, 'article')
results_list = [{'title': i.find_element(By.TAG_NAME, 'a').text,
                 'date': i.find_element(By.XPATH, ".//span[@class='published']").text, 
                 'link': i.find_element(By.TAG_NAME, 'a').get_attribute('href')} for i in results]

df = pd.DataFrame.from_dict(results_list)
    
driver.close()

In [13]:
# add blank content column for later use
df['content'] = np.nan
df

,title,date,link,content
0,Read more about New Laws Going Into Effect On ...,"Dec 29, 2023",https://www.gov.ca.gov/2023/12/29/new-laws-goi...,NaN
1,Read more about California Expands Outdoor Acc...,"Dec 28, 2023",https://www.gov.ca.gov/2023/12/28/california-e...,NaN
2,Read more about Here’s What California Accompl...,"Dec 28, 2023",https://www.gov.ca.gov/2023/12/28/2023-recap/,NaN
3,Read more about Supporting Access to Healthy F...,"Dec 22, 2023",https://www.gov.ca.gov/2023/12/22/supporting-a...,NaN
4,Read more about Making San Francisco Safer: St...,"Dec 22, 2023",https://www.gov.ca.gov/2023/12/22/sf-operation...,NaN


### Scrape all links

In [14]:
# define webpage scraping 
def scrape_california_links(website):
    driver = webdriver.Chrome()
    driver.get(website)

    #wait = WebDriverWait(driver, 3)
    results = driver.find_elements(By.TAG_NAME, 'article')
    results_list = [{'title': i.find_element(By.TAG_NAME, 'a').text,
                 'date': i.find_element(By.XPATH, ".//span[@class='published']").text, 
                 'link': i.find_element(By.TAG_NAME, 'a').get_attribute('href')} for i in results]

    df = pd.DataFrame.from_dict(results_list)
    df['content'] = np.nan
    
    driver.close()
    
    return df

In [15]:
scrape_california_links('https://www.gov.ca.gov/2023/11/')

,title,date,link,content
0,Read more about California Launches Reproducti...,"Nov 29, 2023",https://www.gov.ca.gov/2023/11/29/launches-rep...,NaN
1,Read more about New Report Highlights the Prom...,"Nov 28, 2023",https://www.gov.ca.gov/2023/11/28/new-report-h...,NaN
2,Read more about Governor Newsom Appoints New B...,"Nov 28, 2023",https://www.gov.ca.gov/2023/11/28/governor-new...,NaN
3,Read more about Governor Newsom Announces New ...,"Nov 28, 2023",https://www.gov.ca.gov/2023/11/28/governor-new...,NaN
4,"Read more about California Has Removed 5,679 E...","Nov 27, 2023",https://www.gov.ca.gov/2023/11/27/california-h...,NaN


In [17]:
df_list = []

# Outer loop for 'i' (range 1 to 11)
for i in range(1, 13):  # Loop through 12 pages for 'i'
    
    # Inner loop for 'x'
    x = 1  # Initialize 'x' to 1
    
    while True:  
        try: 
            # Scrape data for the current page
            df = scrape_california_links(f'https://www.gov.ca.gov/2023/{i}/page/{x}/')
            df['page_scraped'] = i 
            df_list.append(df)
            
            # Increment 'x' for the next page
            x += 1
            
        except Exception as e:
            print(f'Error: {e} at page {i}, x={x}')
            break

Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//span[@class='published']"}
  (Session info: chrome=121.0.6167.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104f267dc chromedriver + 4040668
1   chromedriver                        0x0000000104f1e9e0 chromedriver + 4008416
2   chromedriver                        0x0000000104b91870 chromedriver + 284784
3   chromedriver                        0x0000000104bd5080 chromedriver + 561280
4   chromedriver                        0x0000000104bcaee8 chromedriver + 519912
5   chromedriver                        0x0000000104c0f048 chromedriver + 798792
6   chromedriver                        0x0000000104bc974c chromedriver + 513868
7   chromedriver                        0x0000000104bca044 chromedriver + 516164
8   chromedriver                 

Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//span[@class='published']"}
  (Session info: chrome=121.0.6167.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100c3e7dc chromedriver + 4040668
1   chromedriver                        0x0000000100c369e0 chromedriver + 4008416
2   chromedriver                        0x00000001008a9870 chromedriver + 284784
3   chromedriver                        0x00000001008ed080 chromedriver + 561280
4   chromedriver                        0x00000001008e2ee8 chromedriver + 519912
5   chromedriver                        0x0000000100927048 chromedriver + 798792
6   chromedriver                        0x00000001008e174c chromedriver + 513868
7   chromedriver                        0x00000001008e2044 chromedriver + 516164
8   chromedriver                 

Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//span[@class='published']"}
  (Session info: chrome=121.0.6167.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010261e7dc chromedriver + 4040668
1   chromedriver                        0x00000001026169e0 chromedriver + 4008416
2   chromedriver                        0x0000000102289870 chromedriver + 284784
3   chromedriver                        0x00000001022cd080 chromedriver + 561280
4   chromedriver                        0x00000001022c2ee8 chromedriver + 519912
5   chromedriver                        0x0000000102307048 chromedriver + 798792
6   chromedriver                        0x00000001022c174c chromedriver + 513868
7   chromedriver                        0x00000001022c2044 chromedriver + 516164
8   chromedriver                 

In [20]:
# combine into one df
combined = pd.concat(df_list, ignore_index=True)
combined.info()
combined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         608 non-null    object 
 1   date          608 non-null    object 
 2   link          608 non-null    object 
 3   content       0 non-null      float64
 4   page_scraped  608 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 23.9+ KB


,title,date,link,content,page_scraped
0,Read more about Governor Newsom Signs Executiv...,"Jan 31, 2023",https://www.gov.ca.gov/2023/01/31/governor-new...,NaN,1
1,Read more about Governor Newsom Ends Multiple ...,"Jan 31, 2023",https://www.gov.ca.gov/2023/01/31/governor-new...,NaN,1
2,Read more about Governor Newsom Announces Awar...,"Jan 31, 2023",https://www.gov.ca.gov/2023/01/31/governor-new...,NaN,1
3,Read more about Governor Newsom Announces Judi...,"Jan 31, 2023",https://www.gov.ca.gov/2023/01/31/governor-new...,NaN,1
4,Read more about Governor Newsom Announces Thre...,"Jan 31, 2023",https://www.gov.ca.gov/2023/01/31/governor-new...,NaN,1


In [19]:
# save a copy just in case
combined.to_csv('california_links.csv', index = False)

In [ ]:
# combined = pd.read_csv('california_links.csv')
# combined.head()

## Scrape content from links
`Beautiful Soup`

### Test on one link

In [23]:
# define link scraping 
def get_california_content(link, df, header_list):
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                df['content']
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('article').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u' ') for i in string_result]
                result_merged = " ".join(string_result_clean)
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [21]:
# access webpage
link = 'https://www.gov.ca.gov/2023/12/04/rfa-joint-otc-letter-to-hhs/'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [22]:
# parse and isolate content
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('article').find_all('p')
string_result = [str(i.text) for i in result]
string_result_clean = [i.replace(u'\xa0', u' ') for i in string_result]
result_merged = " ".join(string_result_clean)
result_merged

'Published: Dec 04, 2023 SACRAMENTO – Today, Governor Gavin Newsom called on U.S. Health and Human Services Secretary Xavier Becerra to take additional steps to ensure that all forms of contraception are affordable and accessible.  “Accessibility and affordability are essential when it comes to reproductive health care,” said Governor Newsom. “As some parts of the country look to roll back rights, we are standing together with the Biden-Harris administration to protect women and ensure their freedom.”  WHY THIS MATTERS: The non-prescription daily birth control pill could revolutionize access to contraception in the United States — if it is affordable. These governors are joining forces with the Biden Administration in support of affordability by advocating for steps to be taken to require most health plans to cover the cost of the medication. Governor Newsom, joined by the other Executive Committee governors of the Reproductive Freedom Alliance; Illinois Governor JB Pritzker, Massachus

### Scrape content from all links

In [24]:
# test on 5 to see if it works
for i in combined.link[0:5]:
     get_california_content(i, combined, header_list)
        
# combined

'success: content added from https://www.gov.ca.gov/2023/01/31/governor-newsom-signs-executive-order-to-support-storm-response-and-recovery-efforts-1-31-23/'

'success: content added from https://www.gov.ca.gov/2023/01/31/governor-newsom-ends-multiple-open-states-of-emergency/'

'success: content added from https://www.gov.ca.gov/2023/01/31/governor-newsom-announces-awards-of-2-5-billion-for-public-transportation-projects-throughout-california/'

'success: content added from https://www.gov.ca.gov/2023/01/31/governor-newsom-announces-judicial-appointments-1-31-23/'

'success: content added from https://www.gov.ca.gov/2023/01/31/governor-newsom-announces-three-state-sites-identified-for-affordable-transit-oriented-housing-in-sacramento/'

In [25]:
# do for all
for i in combined.link:
     get_california_content(i, combined, header_list)

'content already existing for https://www.gov.ca.gov/2023/01/31/governor-newsom-signs-executive-order-to-support-storm-response-and-recovery-efforts-1-31-23/'

'content already existing for https://www.gov.ca.gov/2023/01/31/governor-newsom-ends-multiple-open-states-of-emergency/'

'content already existing for https://www.gov.ca.gov/2023/01/31/governor-newsom-announces-awards-of-2-5-billion-for-public-transportation-projects-throughout-california/'

'content already existing for https://www.gov.ca.gov/2023/01/31/governor-newsom-announces-judicial-appointments-1-31-23/'

'content already existing for https://www.gov.ca.gov/2023/01/31/governor-newsom-announces-three-state-sites-identified-for-affordable-transit-oriented-housing-in-sacramento/'

...

'success: content added from https://www.gov.ca.gov/2023/12/05/governor-and-first-partner-honor-fallen-airmen-12-5-23/'

'success: content added from https://www.gov.ca.gov/2023/12/05/governor-newsom-and-first-partner-siebel-newsom-to-host-the-92nd-annual-california-state-capitol-virtual-tree-lighting-ceremony/'

'success: content added from https://www.gov.ca.gov/2023/12/04/rfa-joint-otc-letter-to-hhs/'

'success: content added from https://www.gov.ca.gov/2023/12/01/over-40-acres-of-ancestral-land-returned-to-native-american-tribe/'

'success: content added from https://www.gov.ca.gov/2023/12/01/governor-newsom-statement-on-passing-of-u-s-supreme-court-justice-sandra-day-oconnor/'

## Validate and Clean 

In [26]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,date,link,content,page_scraped


In [27]:
# convert date to date-time object to subset
combined['date_clean'] = pd.to_datetime(combined['date'])

In [28]:
# drop original date column 
california = combined.drop(columns=['date']).copy()

In [29]:
# subset to 2023 only 
california_2023 = california[(california['date_clean'] >= '2023-01-01')
                     & (california['date_clean'] <= '2023-12-31')].copy()

In [30]:
print(f"pulled {str(len(california_2023))} documents from california for 2023")

pulled 608 documents from california for 2023


In [34]:
california_2023.head()

,title,link,content,page_scraped,date_clean
0,Read more about Governor Newsom Signs Executiv...,https://www.gov.ca.gov/2023/01/31/governor-new...,"Published: Jan 31, 2023 SACRAMENTO – Governor ...",1,2023-01-31
1,Read more about Governor Newsom Ends Multiple ...,https://www.gov.ca.gov/2023/01/31/governor-new...,"Published: Jan 31, 2023 The text of today’s pr...",1,2023-01-31
2,Read more about Governor Newsom Announces Awar...,https://www.gov.ca.gov/2023/01/31/governor-new...,"Published: Jan 31, 2023 Funding is part of a h...",1,2023-01-31
3,Read more about Governor Newsom Announces Judi...,https://www.gov.ca.gov/2023/01/31/governor-new...,"Published: Jan 31, 2023 SACRAMENTO – Governor ...",1,2023-01-31
4,Read more about Governor Newsom Announces Thre...,https://www.gov.ca.gov/2023/01/31/governor-new...,"Published: Jan 31, 2023 SACRAMENTO – Governor ...",1,2023-01-31


## Export

In [31]:
# export
california_2023.to_csv('california_2023.csv', index = False)

In [ ]:
# test = pd.read_csv('alabama_2023.csv')
# test.head()